
### **4.5 Kernel Thread Context Switching**

**Question:** Describe the actions taken by a kernel to context-switch between kernel-level threads.

**Answer:** Context switching between kernel threads requires the kernel to save the current state of the CPU registers for the thread that is being switched out. It then restores the previously saved register state for the new thread that is being scheduled to run.

### **4.6 Resource Usage: Threads vs. Processes**

**Question:** What resources are used when a thread is created? How do they differ from those used when a process is created?

**Answer:** Thread creation uses significantly fewer resources than process creation. Creating a process requires allocating a large Process Control Block (PCB), which includes a memory map, list of open files, and environment variables. Allocating the memory map is particularly time-consuming. In contrast, creating a thread only involves allocating a small data structure to hold a register set, stack, and priority.

### **4.7 Binding Real-Time Threads to LWPs**

**Question:** Assume that an operating system maps user-level threads to the kernel using the many-to-many model and that the mapping is done through LWPs. Furthermore, the system allows developers to create real-time threads for use in real-time systems. Is it necessary to bind a real-time thread to an LWP? Explain.

**Answer:** Yes, it is necessary to bind a real-time thread to an LWP. For real-time applications, guaranteed and immediate scheduling is critical. If a real-time thread is not bound, it may be forced to wait for an available LWP to become attached to before it can run after being scheduled. This unbounded waiting period is unacceptable for real-time constraints. By binding an LWP to the real-time thread, you ensure the thread can begin execution with minimal delay the moment it is scheduled.


### **4.8 Poor Multithreading Performance Examples**

**Question:** Provide two programming examples in which multithreading does not provide better performance than a single-threaded solution.

**Answer:**
a. Any simple, sequential task where the overhead of thread creation, context switching, and synchronization is greater than the performance benefit of parallel execution. For example, a program that calculates the sum of a small array.
b. A task that is inherently sequential, where each step depends on the result of the previous one, such as computing the Fibonacci sequence recursively or a linked list traversal where each node points to the next.


### **4.10 Shared State in a Multithreaded Process**

**Question:** Which of the following components of program state are shared across threads in a multithreaded process?




a. Register values


b. Heap memory



c. Global variables



d. Stack memory

**Answer:**
b. Heap memory and c. Global variables are shared across threads. Each thread has its own a. Register values and d. Stack memory.





### **4.11 User Threads on a Multiprocessor System**

**Question:** Can a multithreaded solution using multiple user-level threads achieve better performance on a multiprocessor system than on a single-processor system? Explain.

**Answer:** Typically, no. Since the operating system kernel is only aware of the single process and its associated kernel thread, it cannot schedule multiple user-level threads onto different processors concurrently. The entire process, including all its user-level threads, is scheduled on a single processor. Therefore, the parallelism cannot be exploited across multiple CPUs.





### **4.12 Chrome: Processes vs. Threads for Tabs**

**Question:** In Chapter 3, we discussed Google’s Chrome browser and its practice of opening each new tab in a separate process. Would the same benefits have been achieved if, instead, Chrome had been designed to open each new tab in a separate thread? Explain.

**Answer:** No, the key benefits would not be achieved. The main benefits of using separate processes are:
*   **Stability and Isolation:** A crash in one tab (process) does not bring down the entire browser, as processes have separate memory spaces. Threads within a single process share memory, so a bug in one thread could corrupt memory and crash all tabs.
*   **Security:** The operating system can enforce stricter security boundaries (sandboxing) between processes than between threads.







### **4.15 Task vs. Data Parallelism**

**Question:** Determine if the following problems exhibit task or data parallelism:
*   Using a separate thread to generate a thumbnail for each photo in a collection
*   Transposing a matrix in parallel
*   A networked application where one thread reads from the network and another writes to the network
*   The fork-join array summation application described in Section 4.5.2
*   The Grand Central Dispatch system

**Answer:**
*   **Generating thumbnails:** **Data parallelism.** The same task (generate a thumbnail) is applied to different data (each photo).
*   **Transposing a matrix:** **Data parallelism.** Different threads work on different sections (rows/columns/blocks) of the same matrix to perform the identical transposition operation.
*   **Networked application:** **Task parallelism.** The two threads are performing different, specialized tasks (reading and writing).
*   **Fork-join array summation:** **Data parallelism.** The array is split into parts, and the same summation task is performed on each part concurrently.
*   **Grand Central Dispatch:** **Both.** It is a system designed to manage both types. It can execute different, unrelated tasks (task parallelism) and also divide a large dataset to be processed by multiple threads (data parallelism).







### **4.16 Threading Strategy for a CPU-Intensive Application**

**Question:** A system with two dual-core processors has four processors available for scheduling. A CPU-intensive application is running on this system. All input is performed at program start-up, when a single file must be opened. Similarly, all output is performed just before the program terminates, when the program results must be written to a single file. Between start-up and termination, the program is entirely CPU-bound. Your task is to improve the performance of this application by multithreading it. The application runs on a system that uses the one-to-one threading model (each user thread maps to a kernel thread).
- How many threads will you create to perform the input and output? Explain.
- How many threads will you create for the CPU-intensive portion of the application? Explain.

**Answer:**
- **Input and Output:** A single thread for each. Since I/O involves a single file for both operations, using one thread for input and one for output is sufficient. Creating more I/O threads would not speed up access to a single sequential file and could even slow it down due to contention.
- **CPU-intensive portion:** Four threads. Because the system has four processing cores and uses a one-to-one threading model, creating four threads allows the operating system to schedule each thread on a separate core simultaneously, maximizing parallel execution and CPU utilization.

### **4.17 Process and Thread Creation Analysis**

**Question:** Consider the following code segment:
```c
pid_t pid;
pid = fork();
if (pid == 0) { /* child process */
    fork();
    thread_create( . . .);
}
fork();
```
a. How many unique processes are created?
b. How many unique threads are created?

**Answer:**
a. **5 unique processes** are created.
b. **1 unique thread** is created (in addition to the original main thread in that process).

### **4.18 Kernel Process-Thread Models: Linux vs. Windows**

**Question:** As described in Section 4.7.2, Linux does not distinguish between processes and threads. Instead, Linux treats both in the same way, allowing a task to be more akin to a process or a thread depending on the set of flags passed to the clone() system call. However, other operating systems, such as Windows, treat processes and threads differently. Typically, such systems use a notation in which the data structure for a process contains pointers to the separate threads belonging to the process. Contrast these two approaches for modeling processes and threads within the kernel.

**Answer:**
The contrast lies in the fundamental data structure used:
- **Linux's Unified Approach:** Linux uses a single data structure (`task_struct`) to represent both processes and threads. A "process" is a task that does not share most of its resources (like memory maps) with its parent. A "thread" is a task that shares most of its resources (created with the `CLONE_VM`, `CLONE_FS`, and `CLONE_FILES` flags). This design is simpler and more uniform.
- **Windows' Differentiated Approach:** Windows explicitly distinguishes between processes and threads. An `EPROCESS` block represents a process and contains its resource list, while separate `ETHREAD` blocks represent each thread belonging to that process. The `EPROCESS` block points to these threads. This model creates a clearer conceptual and structural separation between the resource container (process) and the units of execution (threads).

### **4.19 Pthreads and Fork Output Analysis**

**Question:** The program shown in the following code uses the Pthreads API. What would be the output from the program at LINE C and LINE P?

```c
#include <pthread.h>
#include <stdio.h>
int value = 0;
void *runner(void *param); /* the thread */

int main(int argc, char *argv[]) {
    pid_t pid;
    pthread_t tid;
    pthread_attr_t attr;
    pid = fork();

    if (pid == 0) { /* child process */
        pthread_attr_init(&attr);
        pthread_create(&tid, &attr, runner, NULL);
        pthread_join(tid, NULL);
        printf("CHILD: value = %d\n", value); /* LINE C */
    }
    else if (pid > 0) { /* parent process */
        wait(NULL);
        printf("PARENT: value = %d\n", value); /* LINE P */
    }
}

void *runner(void *param) {
    value = 5;
    pthread_exit(0);
}
```

**Answer:**
- **LINE C (CHILD):** `value = 5`. The child process creates a thread that changes the value in the child's memory space to 5.
- **LINE P (PARENT):** `value = 0`. The `fork()` call creates a duplicate but separate memory space for the child. The change made by the thread in the child process does not affect the parent process's memory. The parent's `value` remains 0.
